In [ ]:
import librosa
import numpy as np
from tifresi.stft import GaussTruncTF
from tifresi.transforms import log_spectrogram, log_mel_spectrogram, mel_spectrogram
from tifresi.utils import preprocess_signal
import matplotlib.pyplot as plt
from pathlib import Path

class AudioLoader(object):
	def __init__(self, sampling_rate, window_length, hop_size, dynamic_range_dB=50, normalize=True):
		super(AudioLoader, self).__init__()

		self._sampling_rate = sampling_rate
		self._window_length = window_length
		self._hop_size = hop_size
		self._dynamic_range_dB = dynamic_range_dB
		self._normalize = normalize
		self._anStft = GaussTruncTF(hop_size=hop_size, stft_channels=window_length)

	def hopSize(self):
		return self._hop_size

	def windowLength(self):
		return self._window_length

	def loadSound(self, file_name):
		audio, sr = librosa.load(file_name, sr=self._sampling_rate, dtype=np.float64)
		return preprocess_signal(audio)

	def computeSpectrogram(self, audio):
		audio = audio[:len(audio)-np.mod(len(audio), self._window_length)]
		audio = audio[:len(audio)-np.mod(len(audio), self._hop_size)]

		spectrogram = self._anStft.spectrogram(audio)
		logSpectrogram = log_spectrogram(spectrogram, dynamic_range_dB=self._dynamic_range_dB)

		logSpectrogram = logSpectrogram / (self._dynamic_range_dB / 2) + 1
		return logSpectrogram

	def loadAsSpectrogram(self, file_name):
		audio = self.loadSound(file_name)
		return self.computeSpectrogram(audio)

In [ ]:

audioLoader = AudioLoader(22050, 1024, 256, 50)


dataFolder = "../../../Datasets/maestro-v2.0.0/"
dataFolder = "datafolder/"

for filename in Path(dataFolder).rglob('*.wav'):
    audio_ori = audioLoader.loadSound(filename)
    spectrogram_ori = audioLoader.computeSpectrogram(audio_ori)
    break
    


In [ ]:
audio = audio_ori
plt.plot(audio_ori)

In [ ]:
audio.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

sr = 22050

start = 1024*256
length = 480*256
gap = 64*256

sig = np.concatenate([audio[start:start+length], np.zeros(gap), audio[start+gap+length:start+gap+2*length]])
t = (np.arange(len(sig))-length)/sr
plt.figure(dpi=200)
plt.plot(t, sig)
plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off,
    left=False,
    top=False,         # ticks along the top edge are off
    labelleft=False,
    labelbottom=True) # labels along the bottom edge are off
plt.xlim([np.min(t), np.max(t)])
plt.xlabel("time (s)")
plt.savefig('maestro_sample_audio.png', bbox_inches='tight',pad_inches=0)

In [ ]:
sig_gap = audio[start+length: start+length+gap]
t = np.arange(len(sig_gap))/sr
plt.figure(dpi=200)
plt.plot(t, sig_gap)
plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off,
    left=False,
    top=False,         # ticks along the top edge are off
    labelleft=False,
    labelbottom=True) # labels along the bottom edge are off
plt.xlim([np.min(t), np.max(t)])
plt.xlabel("time (s)")
plt.savefig('maestro_sample_audio_gap.png', bbox_inches='tight',pad_inches=0)

In [ ]:
sig_ori = np.concatenate([audio[start:start+gap+2*length]])
t = (np.arange(len(sig_ori))-length)/sr
plt.figure(dpi=200)
plt.plot(t, sig_ori)
plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off,
    left=False,
    top=False,         # ticks along the top edge are off
    labelleft=False,
    labelbottom=True) # labels along the bottom edge are off
plt.xlim([np.min(t), np.max(t)])
plt.xlabel("time (s)")
plt.savefig('maestro_sample_audio_ori.png', bbox_inches='tight',pad_inches=0)

In [ ]:
spectrogram = audioLoader.computeSpectrogram(sig)
spectrogram_ori = audioLoader.computeSpectrogram(sig_ori)

In [ ]:
spectrogram.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
hop_size = 256
overlap = 2

start_spec = 0 +overlap
t_spec = t[overlap:-overlap:hop_size]
length_spec = length//hop_size-2*overlap
gap_spec = gap//hop_size+2*overlap

im = spectrogram[:, start_spec:start_spec+ gap_spec+2*length_spec]
im = np.concatenate([spectrogram[:,start_spec:start_spec+length_spec], -np.ones([spectrogram.shape[0], gap_spec]), spectrogram[:, start_spec+length_spec+gap_spec:start_spec+2*length_spec+gap_spec]], axis=1) 

plt.figure(dpi=200)
plt.imshow(np.flip(im, axis=0),  cmap='inferno')
plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off,
    left=False,
    top=False,         # ticks along the top edge are off
    labelleft=False,
    labelbottom=False) # labels along the bottom edge are off
plt.savefig('maestro_sample_spec.png', bbox_inches='tight',pad_inches=0)

In [ ]:
im_ori = spectrogram_ori[:, start_spec:start_spec+ gap_spec+2*length_spec]


plt.figure(dpi=200)
plt.imshow(np.flip(im_ori, axis=0),  cmap='inferno')
plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off,
    left=False,
    top=False,         # ticks along the top edge are off
    labelleft=False,
    labelbottom=False) # labels along the bottom edge are off
plt.savefig('maestro_sample_spec_ori.png', bbox_inches='tight',pad_inches=0)

In [ ]:
im_ori = spectrogram_ori[:, start_spec+length_spec:start_spec+ gap_spec+length_spec]


plt.figure(dpi=200)
plt.imshow(np.flip(im_ori, axis=0),  cmap='inferno')
plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off,
    left=False,
    top=False,         # ticks along the top edge are off
    labelleft=False,
    labelbottom=False) # labels along the bottom edge are off
plt.savefig('maestro_sample_spec_gap.png', bbox_inches='tight',pad_inches=0)

In [ ]:
n_mels = 80
mel = mel_spectrogram(spectrogram_ori, n_mels=n_mels)

mel1 = mel[:,start_spec:start_spec+length_spec]
mel2 = mel[:,start_spec+length_spec+gap_spec:start_spec+2*length_spec+gap_spec]

plt.figure(dpi=200)
plt.imshow(np.flip(mel1, axis=0),  cmap='inferno')
plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off,
    left=False,
    top=False,         # ticks along the top edge are off
    labelleft=False,
    labelbottom=False) # labels along the bottom edge are off
plt.savefig('maestro_sample_spec_mel_1.png', bbox_inches='tight',pad_inches=0)

In [ ]:
plt.figure(dpi=200)
plt.imshow(np.flip(mel2, axis=0),  cmap='inferno')
plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off,
    left=False,
    top=False,         # ticks along the top edge are off
    labelleft=False,
    labelbottom=False) # labels along the bottom edge are off
plt.savefig('maestro_sample_spec_mel_2.png', bbox_inches='tight',pad_inches=0)